## Import 

In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output
import xlwings as xw

In [6]:
wb = xw.Book('../data/OI_long_vs_short.xlsx')
ws = wb.sheets["Sheet1"]
ws.name = "long_vs_short"

## geting view with +ve and -ve

In [7]:
expiry_sp = 'BANKNIFTY23MAR'
lower_limit  = 38500
upper_limit = 40500

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
expiry_march = expiry_march[(expiry_march.strike >= lower_limit) & (expiry_march.strike <= upper_limit)]
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [8]:

bn_token = 260105
start_dt = "2023-03-24"
end_dt = "2023-03-24"
time_frame = '5minute'
df_expiry_hist_data = pd.DataFrame()
strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
strike_symbol_dict[bn_token] = "BANKNIFTY"

inst_expiry = expiry_march['instrument_token'].unique().tolist() 
for i in inst_expiry:
    df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
    df['strike_type'] = strike_symbol_dict[i]
    df_expiry_hist_data = df_expiry_hist_data.append(df)

# BN data process
df_bn_hist_data = df = pd.DataFrame(kite.historical_data(bn_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
df_bn_hist_data['oi'] = df_bn_hist_data['close']
df_bn_hist_data['strike_type'] = "BANKNIFTY"
df_bn_hist_data['oi_change'] = df_bn_hist_data.groupby(['strike_type']).oi.diff()


In [9]:
# For CE and PE 
# Price	OI
# +	 + -> long buildup
# +	- -> Short covering
# -	+ -> short builup 
# -	- -> long unwinding 
def long_vs_short(price, oi):
    if price > 0  and   oi > 0:
        return 'long_buildup'
    elif price > 0 and oi < 0:
        return 'short_covering'
    elif price <0 and oi > 0:
        return "short_buildup"
    else:
        return "long_unwinding"
    
    query("strike_type == '39500PE'")

In [10]:
## getting top 5 OI for each candle 
df_expiry_hist_data['oi_change'] = df_expiry_hist_data.groupby(['strike_type']).oi.diff()
df_expiry_hist_data['oi_per_change'] = df_expiry_hist_data.groupby(['strike_type']).oi.pct_change()
df_expiry_hist_data['oi_per_change'] = (df_expiry_hist_data['oi_per_change']* 100).round(2)
# df_expiry_hist_data['oi_change'] = df_expiry_hist_data.groupby(['strike_type']).oi_change.cumsum()
df_expiry_hist_data['close_change'] = df_expiry_hist_data.groupby(['strike_type']).close.diff()

df_expiry_hist_data_1 = df_expiry_hist_data.groupby('date').agg(
    {
    "oi_change": lambda x: sorted(x,reverse=True)[0:5]
    }
).reset_index().explode('oi_change')

df_expiry_hist_data_1 = df_expiry_hist_data_1.merge(
    df_expiry_hist_data[['date','close_change','oi_change','oi_per_change','strike_type']], how = 'left',on = ['date','oi_change']
    )
df_expiry_hist_data_1.dropna(inplace=True)
df_expiry_hist_data_1['l_vs_s'] = df_expiry_hist_data_1.apply(lambda x:long_vs_short(x.close_change, x.oi_change),axis =1  )


In [11]:
# oi change-> ek strike price hai 
# 1. change in 5 mins -> prev_oi - current_oi
# trending oi chart -> 2.(  prev_oi - current_oi).sum()


In [12]:
# df_expiry_hist_data.query('strike_type == "39500PE"')

In [13]:
exp_cols = ['date', 'strike_type','oi_change',"oi_per_change",'close_change','l_vs_s']
bn_cols = ['date','oi','strike_type','oi_change']

df_expiry_hist_data_1 = df_expiry_hist_data_1[exp_cols]
df_bn_hist_data = df_bn_hist_data[bn_cols]

df_expiry_hist_data_1.oi_change = df_expiry_hist_data_1.oi_change.astype(int)
df_expiry_hist_data_1 = df_expiry_hist_data_1.merge(df_bn_hist_data, on = 'date').drop(['strike_type_y'],axis = 1)
df_expiry_hist_data_1.loc[df_expiry_hist_data_1['date'].duplicated(), 'date'] = np.NaN


In [14]:
ws.clear_contents()
ws.range("A1").options(index=False).value = df_expiry_hist_data_1
# df_expiry_hist_data_1 = pd.pivot_table(df_expiry_hist_data_1, index =['date'], columns = 'strike_type',)
# df_bn_hist_data.merge(df_expiry_hist_data_1, on = 'date').to_csv('top_5_ls.csv')

## pcr

In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output
import xlwings as xw
from datetime import datetime
# wb = xw.Book('../data/PCR.xlsx')
# ws = wb.sheets["PCR"]
# ws.name = "PCR"

In [2]:
expiry_sp = 'BANKNIFTY23MAR'
lower_limit  = 38500
upper_limit = 40500

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
expiry_march = expiry_march[(expiry_march.strike >= lower_limit) & (expiry_march.strike <= upper_limit)]
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [3]:
def get_pcr(range_pcr, df_bn_hist_data):
    df_bn_hist_data["current_strike"] =( df_bn_hist_data['close'] - (df_bn_hist_data['close']% 100)).astype(int)
    if range_pcr == -1:
        df_bn_hist_data['near_stike_price']= df_bn_hist_data.apply(lambda x:  list(range(df_expiry_hist_data.near_stike_price.min(),df_expiry_hist_data.near_stike_price.max(), 100 )),axis = 1)
    elif range_pcr == 5:
        df_expiry_hist_data
        df_bn_hist_data['near_stike_price']= df_bn_hist_data.apply(lambda x:  list(range(df_expiry_hist_data.near_stike_price.min(),df_expiry_hist_data.near_stike_price.max(), 100 )),axis = 1)
    else:
        df_bn_hist_data['near_stike_price']= df_bn_hist_data.apply(lambda x:  list(range(x.current_strike- range_pcr,x.current_strike + range_pcr, 100 )),axis = 1)
    
    df_bn_hist_data  = df_bn_hist_data.explode("near_stike_price")
    df_merged = df_bn_hist_data[['date','near_stike_price']].merge(
                            df_expiry_hist_data[['date','near_stike_price','oi','strike_type']], how = 'left', on =['date','near_stike_price'] )
    df_merged["call_put"] = df_merged.strike_type.str[-2:]
    df_merged = df_merged.groupby(['date','call_put']).oi.sum().reset_index()
    df_merged = pd.pivot_table(df_merged,index ='date', columns='call_put')
    df_merged.columns = ['_'.join(i) for i in df_merged.columns]
    df_merged[f'PCR_{range_pcr}'] = df_merged['oi_PE']/ df_merged['oi_CE']
    df_merged = df_merged.round(2)
    return df_merged.reset_index()

In [4]:

bn_token = 260105
start_dt = "2023-03-27"
end_dt = "2023-03-27"
time_frame = '5minute'
while True:
    if datetime.now().minute % 5 == 1:
        print(datetime.now())
        df_expiry_hist_data = pd.DataFrame()
        strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
        strike_symbol_dict[bn_token] = "BANKNIFTY"

        # stike price data fetch 
        inst_expiry = expiry_march['instrument_token'].unique().tolist() 
        for i in inst_expiry:
            df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
            df['strike_type'] = strike_symbol_dict[i]
            df_expiry_hist_data = df_expiry_hist_data.append(df)

        df_expiry_hist_data['near_stike_price'] = df_expiry_hist_data.strike_type.str[:-2].astype(int)

        # BN data process
        df_bn_hist_data = df = pd.DataFrame(kite.historical_data(bn_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
        df_bn_hist_data['oi'] = df_bn_hist_data['close']
        df_bn_hist_data['strike_type'] = "BANKNIFTY"
        # df_bn_hist_data['oi_change'] = df_bn_hist_data.groupby(['strike_type']).oi.diff()

        
        # df_combined_1 = df_expiry_hist_data.groupby('date').agg(
        #     {
        #     "oi": lambda x: sorted(x,reverse=True)[0:5]
        #     }
        # ).reset_index().explode('oi_change')

        df_pcr_5 = get_pcr(500, df_bn_hist_data)
        df_pcr_1 = get_pcr(100, df_bn_hist_data)
        df_pcr_all = get_pcr(-1, df_bn_hist_data)
        df_pcr_top5 = get_pcr(5, df_bn_hist_data)
        
        fig = go.Figure()
        data1 = []
        data2 = []
        data3 = []
        # trace_bn = go.Scatter(x=df_bn_hist_data.date,
        #                                         y=df_bn_hist_data["close"],
        #                                         name = "BANKNIFTY",
        #                                         yaxis='y2'
        #                                         )

        trace_bn = go.Candlestick(x=df_bn_hist_data.date,
                        open=df_bn_hist_data.open,
                        high=df_bn_hist_data.high,
                        low=df_bn_hist_data.low,
                        close=df_bn_hist_data.close,
                        name = "BN",
                            yaxis='y2'
                            )
        data1.append(go.Scatter(x=df_pcr_1.date,
                                                y=df_pcr_1["PCR_100"],
                                                name = "pcr_1",
                                                yaxis='y1'
                                                ))


        data2.append(go.Scatter(x=df_pcr_5.date,
                                                y=df_pcr_5["PCR_500"],
                                                name = "pcr_5",
                                                yaxis='y1'
                                                ))
        data3.append(go.Scatter(x=df_pcr_all.date,
                                                y=df_pcr_all["PCR_-1"],
                                                name = "pcr_all",
                                                yaxis='y1'
                                                ))

        data1.append(trace_bn)
        data2.append(trace_bn)
        data3.append(trace_bn)

        layout = go.Layout(title='OI vs Bank',
                                yaxis=dict(title='Oi'),
                                yaxis2=dict(title='bank',
                                            overlaying='y',
                                            side='right'),
                                            legend={'traceorder':'normal'}
                                            )
        fig1 = go.Figure(data=data1, layout=layout)
        fig2 = go.Figure(data=data2, layout=layout)
        fig3 = go.Figure(data=data3, layout=layout)
        fig1.update_layout(height=500, width=1200)
        fig2.update_layout(height=500, width=1200)
        fig3.update_layout(height=500, width=1200)
        fig1.show()
        fig2.show()
        fig3.show()
        clear_output(wait=True)   
        time.sleep(60)


2023-03-27 13:26:00.000588


KeyError: 'data'

In [ ]:
# 1. above 5 and below 5 pe 
# 2 above and 1 below + stirke
# 3. overall PCR
# 4. top 5 PCR
# 5. close on secondary axis  

In [16]:
# ws.clear_contents()
# ws.range("A1").options(index=True).value = df_merged